<table align="left">
  <tr>
    <td><img src="fleet.png" alt="fleet of icecream trucks" width="120"/></td>
    <td align="left"><h1>Lesson 4: Monitoring and Evaluating your Agent</h1></td>
  </tr>
</table>


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> file:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Setup Tracing

In [1]:
PROJECT_NAME = "Customer-Success"

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
from phoenix.otel import register
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

tracer_provider = register(
    project_name=PROJECT_NAME,
    #endpoint= get_phoenix_endpoint() + "v1/traces"
    endpoint = os.getenv('DLAI_LOCAL_URL').format(port='6006') + "v1/traces"
)
SmolagentsInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: Customer-Success
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://s172-29-40-229p6006.lab-aws-production.deeplearning.ai/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv() # load variables from local .env file

from huggingface_hub import login

login(os.getenv('HF_API_KEY'))

In [4]:
from smolagents import HfApiModel

model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

model([{"role": "user", "content": "Hello!"}])

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Hello! How can I assist you today?', tool_calls=[], raw=ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='Hello! How can I assist you today?', tool_call_id=None, tool_calls=[]), logprobs=None, seed=13618980564928662000)], created=1745329835, id='nr7pchw-4yUbBN-9345974f3e93058b', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint=None, usage=ChatCompletionOutputUsage(completion_tokens=10, prompt_tokens=31, total_tokens=41), object='chat.completion', prompt=[]))

In [5]:
# This is where you can access the display:
print(os.environ.get('DLAI_LOCAL_URL').format(port='6006'))

https://s172-29-40-229p6006.lab-aws-production.deeplearning.ai/


## Trace an agent run

In [6]:
from smolagents import HfApiModel, CodeAgent

agent = CodeAgent(model=model, tools=[])

>Note, the following line will sometimes get a timeout on the interface to the tracing package due to the networked interface. If this happens, try it again.


In [7]:
agent.run("What is the 100th Fibonacci number?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the 100th Fibonacci number?                                                                             │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def fibonacci(n):                                                                                                
      a, b = 0, 1                                                                                                  
      for _ in range(n):                                                                                           
          a, b = b, a + b                                                                                          
      return a                                                                                                     
                                                                                                                   
  fib_100 = fibonacci(100)                                                                                         
  final_answer(fib_100)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 354224848179261915075

[Step 1: Duration 0.12 seconds| Input tokens: 2,020 | Output tokens: 130]

354224848179261915075

In [8]:
# This is where you can access the display:
print(os.environ.get('DLAI_LOCAL_URL').format(port='6006'))

https://s172-29-40-229p6006.lab-aws-production.deeplearning.ai/


## Setup ice cream production system

In [9]:
from smolagents import tool
from typing import Dict

menu_prices = {"crepe nutella": 1.50, "vanilla ice cream": 2, "maple pancake": 1.}

ORDER_BOOK = {}

@tool
def place_order(quantities: Dict[str, int], session_id: int) -> None:
    """Places a pre-order of snacks.

    Args:
        quantities: a dictionary with names as keys and quantities as values
        session_id: the id for the client session
    """
    global ORDER_BOOK
    assert isinstance(quantities, dict), "Incorrect type for the input dictionary!"
    assert [key in menu_prices for key in quantities.keys()], f"All food names should be within {menu_prices.keys()}"
    ORDER_BOOK[session_id] = quantities

@tool
def get_prices(quantities: Dict[str, int]) -> str:
    """Gets price for certain quantities of ice cream.

    Args:
        quantities: a dictionary with names as keys and quantities as values
    """
    assert isinstance(quantities, dict), "Incorrect type for the input dictionary!"
    assert [key in menu_prices for key in quantities.keys()], f"All food names should be within {menu_prices.keys()}"
    total_price = sum([menu_prices[key] * value for key, value in quantities.items()])
    return (
        f"Given the current menu prices:\n{menu_prices}\nThe total price for your order would be: ${total_price}"
    )

In [10]:
order_agent = CodeAgent(
    tools=[place_order, get_prices],
    model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")
)

In [11]:
order_agent.run(
    "Could I come and collect one crepe nutella?",
    additional_args={"session_id": 192}
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could I come and collect one crepe nutella?                                                                     │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 192}.                                                                                            │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  place_order(quantities={'crepe nutella': 1}, session_id=session_id)                                              
  final_answer("Order placed for one crepe nutella.")                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Order placed for one crepe nutella.

[Step 1: Duration 0.11 seconds| Input tokens: 2,193 | Output tokens: 68]

'Order placed for one crepe nutella.'

### Try multiple orders

In [12]:
client_requests = [
    ("Could I come and collect one crepe nutella?", "place_order"),
    ("What would be the price for 1 crêpe nutella + 2 pancakes?", "get_prices"),
    ("How did you start your ice-cream business?", None),
    ("What's the weather at the Louvre right now?", None),
    ("I'm not sure if I should order. I want a vanilla ice cream. but if it's more expensive than $1, I don't want it. If it's below, I'll order it, please.", "place_order")
]

In [13]:
for request in client_requests:
    order_agent.run(
        request[0],
        additional_args={"session_id": 0, "menu_prices": menu_prices}
    )

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could I come and collect one crepe nutella?                                                                     │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 0, 'menu_prices': {'crepe nutella': 1.5, 'vanilla ice cream': 2, 'maple pancake': 1.0}}.         │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  quantities = {'crepe nutella': 1}                                                                                
  place_order(quantities=quantities, session_id=session_id)                                                        
  final_answer("Order placed for one crepe nutella.")                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Order placed for one crepe nutella.

[Step 1: Duration 0.11 seconds| Input tokens: 2,227 | Output tokens: 93]

...

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I'm not sure if I should order. I want a vanilla ice cream. but if it's more expensive than $1, I don't want    │
│ it. If it's below, I'll order it, please.                                                                       │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 0, 'menu_prices': {'crepe nutella': 1.5, 'vanilla ice cream': 2, 'maple pancake': 1.0}}.         │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  vanilla_ice_cream_price = menu_prices['vanilla ice cream']                                                       
  print(f"The price of vanilla ice cream is ${vanilla_ice_cream_price}.")                                          
                                                                                                                   
  final_answer("I will not order the vanilla ice cream as it is more than $1.")                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The price of vanilla ice cream is $2.

Out - Final answer: I will not order the vanilla ice cream as it is more than $1.

[Step 1: Duration 0.08 seconds| Input tokens: 2,260 | Output tokens: 162]

In [14]:
import phoenix as px

spans = px.Client().get_spans_dataframe(project_name=PROJECT_NAME)
spans.head(20)

/usr/local/lib/python3.11/site-packages/phoenix/utilities/client.py:60: UserWarning: The Phoenix server (8.27.1) and client (8.19.1) versions are mismatched and may have compatibility issues.
  warnings.warn(


,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.llm.token_count.completion,attributes.output.value,attributes.openinference.span.kind,attributes.llm.output_messages,attributes.llm.token_count.prompt,attributes.llm.input_messages,attributes.tool.name,attributes.tool.parameters,attributes.tool.description,attributes.smolagents
context.span_id,,,,,,,,,,,,,,,,,,,,,
b9c3a055c281ab45,HfApiModel.__call__,LLM,None,2025-04-25 18:17:03.550788+00:00,2025-04-25 18:17:03.733259+00:00,OK,,[],b9c3a055c281ab45,25337ed963cb454ed8f66d18e8ab965c,...,10.0,"{""role"": ""assistant"", ""content"": ""Hello! How c...",LLM,"[{'message.role': 'assistant', 'message.conten...",31.0,None,None,None,None,None
e010cecb4de94f3d,HfApiModel.__call__,LLM,40e9cb3ac2f13063,2025-04-25 18:17:03.832824+00:00,2025-04-25 18:17:03.857126+00:00,OK,,[],e010cecb4de94f3d,51876544ebbcbe7260dd62a81b9009c3,...,130.0,"{""role"": ""assistant"", ""content"": ""Thought: To ...",LLM,"[{'message.role': 'assistant', 'message.conten...",2020.0,"[{'message.role': 'system', 'message.content':...",None,None,None,None
3c72acf2aca74b06,FinalAnswerTool,TOOL,40e9cb3ac2f13063,2025-04-25 18:17:03.903674+00:00,2025-04-25 18:17:03.903741+00:00,OK,,[],3c72acf2aca74b06,51876544ebbcbe7260dd62a81b9009c3,...,NaN,None,TOOL,None,NaN,None,final_answer,"{'answer': {'type': 'any', 'description': 'The...",Provides a final answer to the given problem.,None
40e9cb3ac2f13063,Step 1,CHAIN,01c820fdd4dab4c7,2025-04-25 18:17:03.832607+00:00,2025-04-25 18:17:03.928199+00:00,OK,,[],40e9cb3ac2f13063,51876544ebbcbe7260dd62a81b9009c3,...,NaN,Execution logs:\nLast output from code snippet...,CHAIN,None,NaN,None,None,None,None,None
01c820fdd4dab4c7,CodeAgent.run,AGENT,None,2025-04-25 18:17:03.824719+00:00,2025-04-25 18:17:03.947593+00:00,OK,,[],01c820fdd4dab4c7,51876544ebbcbe7260dd62a81b9009c3,...,130.0,354224848179261915075,AGENT,None,2020.0,None,None,None,None,"{'max_steps': 20, 'tools_names': ['final_answe..."
57112b07324556c3,HfApiModel.__call__,LLM,205c16c0c40205c5,2025-04-25 18:17:04.001870+00:00,2025-04-25 18:17:04.027422+00:00,OK,,[],57112b07324556c3,e4cd80d1bc8d8db918dac6d6254907e2,...,68.0,"{""role"": ""assistant"", ""content"": ""Thought: I n...",LLM,"[{'message.role': 'assistant', 'message.conten...",2193.0,"[{'message.role': 'system', 'message.content':...",None,None,None,None
f95983764d778b62,SimpleTool,TOOL,205c16c0c40205c5,2025-04-25 18:17:04.048559+00:00,2025-04-25 18:17:04.048612+00:00,OK,,[],f95983764d778b62,e4cd80d1bc8d8db918dac6d6254907e2,...,NaN,None,TOOL,None,NaN,None,place_order,"{'quantities': {'type': 'object', 'additionalP...",Places a pre-order of snacks.,None
d78d6e92ff132df7,FinalAnswerTool,TOOL,205c16c0c40205c5,2025-04-25 18:17:04.067714+00:00,2025-04-25 18:17:04.067768+00:00,OK,,[],d78d6e92ff132df7,e4cd80d1bc8d8db918dac6d6254907e2,...,NaN,None,TOOL,None,NaN,None,final_answer,"{'answer': {'type': 'any', 'description': 'The...",Provides a final answer to the given problem.,None
205c16c0c40205c5,Step 1,CHAIN,8da70f2337dcf3e2,2025-04-25 18:17:04.001652+00:00,2025-04-25 18:17:04.087800+00:00,OK,,[],205c16c0c40205c5,e4cd80d1bc8d8db918dac6d6254907e2,...,NaN,Execution logs:\nLast output from code snippet...,CHAIN,None,NaN,None,None,None,None,None


### Add processing to extract desired information

In [15]:
import pandas as pd
import json

agents = spans[spans['span_kind'] == 'AGENT'].copy()
agents['task'] = agents['attributes.input.value'].apply(
    lambda x: json.loads(x).get('task') if isinstance(x, str) else None
)

tools = spans.loc[
    spans['span_kind'] == 'TOOL',
    ["attributes.tool.name", "attributes.input.value", "context.trace_id"]
].copy()

tools_per_task = agents[
    ["name", "start_time", "task", "context.trace_id"]
].merge(
    tools,
    on="context.trace_id",
    how="left",
)
tools_per_task.head()

,name,start_time,task,context.trace_id,attributes.tool.name,attributes.input.value
0,CodeAgent.run,2025-04-25 18:17:03.824719+00:00,What is the 100th Fibonacci number?,51876544ebbcbe7260dd62a81b9009c3,final_answer,"{""args"": [354224848179261915075], ""sanitize_in..."
1,CodeAgent.run,2025-04-25 18:17:03.996698+00:00,Could I come and collect one crepe nutella?,e4cd80d1bc8d8db918dac6d6254907e2,place_order,"{""args"": [], ""sanitize_inputs_outputs"": false,..."
2,CodeAgent.run,2025-04-25 18:17:03.996698+00:00,Could I come and collect one crepe nutella?,e4cd80d1bc8d8db918dac6d6254907e2,final_answer,"{""args"": [""Order placed for one crepe nutella...."
3,CodeAgent.run,2025-04-25 18:17:05.818822+00:00,Could I come and collect one crepe nutella?,738324ce386f14b12f5c0334db40aecc,place_order,"{""args"": [], ""sanitize_inputs_outputs"": false,..."
4,CodeAgent.run,2025-04-25 18:17:05.818822+00:00,Could I come and collect one crepe nutella?,738324ce386f14b12f5c0334db40aecc,final_answer,"{""args"": [""Order placed for one crepe nutella...."


### Now, compare tool calls with exected tool calls

In [ ]:
def score_request(expected_tool: str, tool_calls: list):
    if expected_tool is None:
        return tool_calls == set(["final_answer"])
    else:
        return expected_tool in tool_calls

results = []
for request, expected_tool in client_requests:
    tool_calls = set(tools_per_task.loc[tools_per_task["task"] == request, "attributes.tool.name"].tolist())
    results.append(
        {
            "request": request,
            "tool_calls_performed": tool_calls,
            "is_correct": score_request(expected_tool, tool_calls)
        }
    )
pd.DataFrame(results)